# Demo af `request`

Et python lib til http-requests

Jeg har nogle filer fra mit løbe ur, med geo-punkter, men højde data er meget uprecise, så jeg vil have noget bedre.
Derfor vil jeg hente fra __opentopodata.org__.

In [ ]:
pip install requests

In [ ]:
import requests
res = requests.get('https://api.opentopodata.org/v1/test-dataset?locations=55.73197203068785,12.343950949062537')
print(res.text)


Det kan jeg godt gøre lidt pænere

In [ ]:
import time

# https://api.opentopodata.org/v1/eudem25m?locations=55.732009,12.343829

lat = 55.921015329658985
long = 11.662305062636733

params = {'locations': f"{lat},{long}"}
url  = 'https://api.opentopodata.org/v1/eudem25m'

r = requests.get(url, params=params)

print(r)
print(r.text)

print(r.json()['results'][0]['elevation'])

Det var noget med nogle filer...

De er i et propriotært binært format `.FIT`, så det skal indlæses med 

In [ ]:
pip install pytz fitdecode

In [ ]:
import pytz
from fit_file import read
    
fname = "data/hok_klubmesterskab_2022/CA8D1347.FIT"
points = read.read_points(fname)
print(points)


In [ ]:
import json
json_points = json.dumps(points, default=str, indent=2)
print(json_points)

In [ ]:
import time

for p in points:
    lat = p['latitude']
    long = p['longitude']

    params = {'locations': f"{lat},{long}"}
    url  = 'https://api.opentopodata.org/v1/eudem25m'

    r = requests.get(url, params=params)

    print(r)
    print(r.text)

    print(r.json()['results'][0]['elevation'])

    p['elevation'] = r.json()['results'][0]['elevation']
    time.sleep(1)

In [ ]:
n = len(points)
wz = 100 # window size

for w in range(0, n, wz):
    # print(w)
    # for i in range(w, w+wz):
    #     print(f"{i},", end=" ")
    
    # print(points[w: w+wz])

    # for p in points[w: w+100]:
    #     print(p)

    lat_long_str_list = [f"{p['latitude']},{p['longitude']}" for p in points[w: w + wz]]
    lat_log_str = "|".join(lat_long_str_list)
    # print(lat_log_str)

    params = {'locations': lat_log_str}
    url  = 'https://api.opentopodata.org/v1/eudem25m'
    r = requests.get(url, params=params)
    if r.status_code != 200:
        print(r.status_code)
        print(r.text)
    results = r.json()['results']
    # print(results)

    for i, ep in enumerate(results):
        points[w + i]['elevation'] = ep['elevation']

    time.sleep(0.5)

points


Forskellen mellem den oprindelige `altitude` og den nytilføjede `elevation`?

In [ ]:
akkumuleret_diff = 0
for p in points:
    print(f"Forskellen mellem `altitude` {p['altitude']} og `elevation` {p['elevation']}: {p['altitude'] - p['elevation']} ")
    akkumuleret_diff += (p['elevation'] - p['altitude'])

akkumuleret_diff / len(points)


Således gemmes som _komma-separerede-værdier_ (csv):

In [52]:
import csv
foutname = fname.split('.')[-2] # næstsidste inden sidste '.'
foutname += '.csv'

with open(foutname, "+w", newline='') as csvfile:
    fieldnames = points[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames)
    writer.writeheader()
    writer.writerows(points)


In [ ]:

import requests

response = requests.get(
    url='https://api.energidataservice.dk/dataset/ElectricityProdex5MinRealtime?offset=0&start=2024-01-01T00:00&sort=Minutes5UTC%20DESC&limit=1000')

result = response.json()

for k, v in result.items():
    print(k, v)

records = result.get('records', [])

print('records:')
# for record in records:
#     print(' ', record)

In [36]:
for p in points:
    print(f"forskellen mellem 'altitude' {p['altitude']} og 'elevation' {p['elevation']} er {p['altitude'] - p['elevation']}")

forskellen mellem 'altitude' 12.799999999999955 og 'elevation' 14.055695533752441 er -1.2556955337524869
forskellen mellem 'altitude' 12.799999999999955 og 'elevation' 14.116682052612305 er -1.3166820526123502
forskellen mellem 'altitude' 12.799999999999955 og 'elevation' 14.143874168395996 er -1.3438741683960416
forskellen mellem 'altitude' 12.799999999999955 og 'elevation' 14.13538932800293 er -1.3353893280029752
forskellen mellem 'altitude' 12.600000000000023 og 'elevation' 14.136137008666992 er -1.5361370086669695
forskellen mellem 'altitude' 12.600000000000023 og 'elevation' 14.13679313659668 er -1.536793136596657
forskellen mellem 'altitude' 12.600000000000023 og 'elevation' 14.136500358581543 er -1.5365003585815202
forskellen mellem 'altitude' 12.600000000000023 og 'elevation' 14.137676239013672 er -1.5376762390136491
forskellen mellem 'altitude' 12.600000000000023 og 'elevation' 14.159758567810059 er -1.5597585678100359
forskellen mellem 'altitude' 12.600000000000023 og 'elevat

In [37]:
import csv
foutname = fname.split('.')[-2]
foutname += '.csv' 

with open(foutname, 'w', newline='') as csvfile:
    fieldnames = points[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames)
    writer.writeheader()
    writer.writerows(points)

In [27]:
import requests 
import pandas as pd

response = requests.get(
    url='https://api.energidataservice.dk/dataset/ElectricityProdex5MinRealtime?offset=0&sort=Minutes5UTC%20DESC')

result = response.json()

for k, v in result.items():
    print(k, v)

records = result.get('records', [])

print(records)

print(len(records))

energy_df = pd.DataFrame(records)

print(energy_df)

total 2066770
sort Minutes5UTC DESC
limit 100
dataset ElectricityProdex5MinRealtime
records [{'Minutes5UTC': '2024-10-28T11:25:00', 'Minutes5DK': '2024-10-28T12:25:00', 'PriceArea': 'DK1', 'ProductionLt100MW': 233.880005, 'ProductionGe100MW': 283.829987, 'OffshoreWindPower': 620.059998, 'OnshoreWindPower': 574.030029, 'SolarPower': 214.970001, 'ExchangeGreatBelt': -449.25, 'ExchangeGermany': 24.440001, 'ExchangeNetherlands': -0.09, 'ExchangeGreatBritain': 0.0, 'ExchangeNorway': 1031.359985, 'ExchangeSweden': 599.0, 'BornholmSE4': None}, {'Minutes5UTC': '2024-10-28T11:25:00', 'Minutes5DK': '2024-10-28T12:25:00', 'PriceArea': 'DK2', 'ProductionLt100MW': 173.339996, 'ProductionGe100MW': 453.790009, 'OffshoreWindPower': 526.440002, 'OnshoreWindPower': 165.210007, 'SolarPower': 182.080002, 'ExchangeGreatBelt': 449.25, 'ExchangeGermany': -840.349976, 'ExchangeNetherlands': None, 'ExchangeGreatBritain': None, 'ExchangeNorway': None, 'ExchangeSweden': 822.570007, 'BornholmSE4': 11.67}, {'Minut